In [774]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [794]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 80

In [776]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [777]:
def select_season (df, season):
    return df[df['season']==season]

In [778]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [779]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [780]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [781]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

   
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test

In [782]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #print(method.score(X_test, Y_test))
        #print('hi')
        lis.append(float(method.score(X_test, Y_test)))

    tmp[method_name]=lis 
    
    return tmp

In [783]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [784]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [785]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

In [793]:
def pred_all_meth (df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction):
    
    lis=[]
    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'
    
    svm_linear = svm.SVC(kernel='linear')
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear_3')))    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'
    
    svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf_3')))

    
    
    #column_to_use_as_prediction = [22,18,51,49,42,20]
    #column_to_predict = 'Goal/No_Goal'  
               
    dtc = DecisionTreeClassifier()
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc_3')))

    
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'  
               
    bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb_3')))          
               
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'           
               
    gradb = GradientBoostingClassifier(max_depth=10, n_estimators=20)
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb_3')))         
   
                    
    
    #column_to_use_as_prediction=[22,18,51,49,42,20]
    #column_to_predict='Goal/No_Goal'  
                    
    Ardf = AdaBoostClassifier(RandomForestClassifier(max_depth=10, n_estimators=20))
    lis.append(MyDf(Predict_whole_season_by_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_1')))
    lis.append(MyDf(Predict_whole_season_with_one(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, Ardf,'rdf_2')))
    #lis.append(MyDf(Predict_season_with_current_season(df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf_3')))
    
    
    
    return lis

In [787]:
def Put_Togheter (lis):
    tmp=pd.DataFrame()
    for i in range(len(lis)):
        tmp=tmp.join(lis[i].df, how='outer')
        
    return tmp

# Loading data

In [796]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [797]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [798]:
n=4
m=4

In [799]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [800]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


# Parameter

In [807]:
season_to_be_predicted='2015/2016'
column_to_predict='FTR'
column_to_use_as_prediction = [65,66]


# RUN with all method only ONE LEAGUE

In [808]:
a=pred_all_meth(allchampionships[1].df, season_to_be_predicted,column_to_predict,column_to_use_as_prediction)

## Put together

In [816]:
b=Put_Togheter(a)

In [817]:
b

,svm_linear_1,svm_linear_2,svm_rbf_1,svm_rbf_2,dtc_1,dtc_2,bagb_1,bagb_2,gradb_1,gradb_2,rdf_1,rdf_2
0,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000
1,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
2,0.250000,0.250000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000,0.500000
3,0.500000,0.500000,0.500000,0.500000,0.375000,0.375000,0.375000,0.375000,0.375000,0.500000,0.500000,0.500000
4,0.500000,0.500000,0.250000,0.250000,0.250000,0.250000,0.250000,0.375000,0.250000,0.250000,0.375000,0.250000
5,0.500000,0.500000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.500000,0.375000,0.375000
6,0.375000,0.375000,0.625000,0.625000,0.625000,0.625000,0.625000,0.625000,0.625000,0.625000,0.625000,0.625000
7,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000,0.250000
8,0.250000,0.250000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000,0.375000
9,0.250000,0.250000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000,0.125000


In [818]:
b.mean()

svm_linear_1    0.373751
svm_linear_2    0.373751
svm_rbf_1       0.355747
svm_rbf_2       0.351117
dtc_1           0.350456
dtc_2           0.355232
bagb_1          0.359862
bagb_2          0.364491
gradb_1         0.350970
gradb_2         0.359862
rdf_1           0.364345
rdf_2           0.361038
dtype: float64

## Get Means and Best Methods

In [773]:
for i in range(len(b.mean()[b.mean()==max(b.mean())])):
    print ('Best method are', b.mean()[b.mean()==max(b.mean())].index[i], 'with score', b.mean()[b.mean()==max(b.mean())][i])


('Best method are', 'svm_linear_1', 'with score', 0.37375073486184601)
('Best method are', 'svm_linear_2', 'with score', 0.37375073486184601)
('Best method are', 'svm_rbf_1', 'with score', 0.37375073486184601)
('Best method are', 'svm_rbf_2', 'with score', 0.37375073486184601)


# Run all methods ALL LEAGUES 

In [683]:
def FINAL (df,season_to_be_predicted):
    lis=[]
    tmp=pd.DataFrame(columns=[u'svm_linear_1', u'svm_linear_2', u'svm_linear_3', u'svm_rbf_1',
       u'svm_rbf_2', u'svm_rbf_3', u'dtc_1', u'dtc_2', u'dtc_3', u'bagb_1',
       u'bagb_2', u'bagb_3', u'gradb_1', u'gradb_2', u'gradb_3', u'rdf_1',
       u'rdf_2', u'rdf_3'])
    
    for i in range(len(df)):
        #print(i)
        #print(allchampionships[i].df['country_id'].iloc[0])
        
        a=(pred_all_meth(df[i].df, season_to_be_predicted))
        b=Put_Togheter(a)
        #print(b.mean())
        tmp=tmp.append(b.mean(), ignore_index=True)
        #print(tmp)
    
    '''tmp.index=['Switzerland Super League', 'Poland Ekstraklasa',
       'Scotland Premier League', 'France Ligue 1',
       'Germany 1. Bundesliga', 'England Premier League',
       'Belgium Jupiler League', 'Portugal Liga ZON Sagres',
       'Netherlands Eredivisie', 'Italy Serie A', 'Spain LIGA BBVA']
    '''
    return tmp
        
        
    

In [684]:
G=FINAL(allchampionships,season_to_be_predicted)

In [685]:
G

,svm_linear_1,svm_linear_2,svm_linear_3,svm_rbf_1,svm_rbf_2,svm_rbf_3,dtc_1,dtc_2,dtc_3,bagb_1,bagb_2,bagb_3,gradb_1,gradb_2,gradb_3,rdf_1,rdf_2,rdf_3
0,0.516241,0.573384,0.540248,0.601956,0.601956,0.540174,0.443878,0.511395,0.541958,0.530187,0.466837,0.557372,0.459949,0.556293,0.519029,0.512245,0.500765,0.536338
1,0.444004,0.444004,0.531147,0.444004,0.444004,0.523753,0.532848,0.516681,0.477403,0.574148,0.505071,0.479939,0.593327,0.573486,0.511577,0.538139,0.574148,0.512770
2,0.562063,0.551468,0.512917,0.431905,0.432302,0.487619,0.502460,0.499286,0.508384,0.480754,0.488690,0.511707,0.501587,0.499246,0.496138,0.494762,0.473175,0.502692
3,0.533519,0.489630,0.516318,0.482593,0.478889,0.494568,0.469630,0.497037,0.503536,0.514444,0.447037,0.494872,0.483333,0.465370,0.509294,0.465741,0.517222,0.509540
4,0.521368,0.521368,0.545349,0.521368,0.521368,0.545894,0.482906,0.457265,0.484226,0.525641,0.495726,0.499228,0.508547,0.512821,0.509494,0.461538,0.482906,0.513646
5,0.465556,0.477593,0.516283,0.500463,0.492963,0.522129,0.516667,0.479074,0.517968,0.499907,0.541019,0.496485,0.497222,0.539444,0.509299,0.511389,0.511852,0.535559
6,0.576923,0.576923,0.532906,0.576923,0.576923,0.540673,0.480769,0.524038,0.507004,0.519231,0.471154,0.516324,0.514423,0.514423,0.493905,0.514423,0.524038,0.530967
7,0.528244,0.506527,0.516421,0.515074,0.515074,0.524553,0.524820,0.513087,0.504832,0.507243,0.509923,0.512098,0.470474,0.457653,0.519691,0.531191,0.487124,0.523467
8,0.564103,0.564103,0.545487,0.564103,0.564103,0.549570,0.512821,0.487179,0.498599,0.542735,0.495726,0.521962,0.534188,0.555556,0.512981,0.525641,0.581197,0.525173
9,0.463333,0.446667,0.484375,0.546667,0.543333,0.491723,0.476667,0.463333,0.492160,0.506667,0.490000,0.496333,0.476667,0.486667,0.486853,0.463333,0.473333,0.494766


## Get The best Methods and Scores for Each League

In [ ]:
for i in range(len(G)):
    
    for i in range(G.iloc[i])
    print ('Best method are', b.mean()[b.mean()==max(b.mean())].index[i], 'with score', b.mean()[b.mean()==max(b.mean())][i])

    print ('Best Methods for' )
    

## SVM_LINEAR

In [495]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [496]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [497]:
c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [498]:
a['svm_linear'].mean()

0.5162414965986394

In [499]:
b['svm_linear'].mean()

0.5733843537414965

In [500]:
c['svm_linear'].mean()

0.5402483852432872

## SVM_RBF


In [501]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [502]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [503]:
a['svm_rbf'].mean()

0.601955782312925

In [504]:
b['svm_rbf'].mean()

0.601955782312925

## CART

In [505]:
dtc = DecisionTreeClassifier()
d=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

e=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

t=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')


In [507]:
d['dtc'].mean()

0.43792517006802717

In [508]:
e['dtc'].mean()

0.5260204081632655

In [509]:
#a.join(e, how='outer')

# Adaboost Classifier 

In [510]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')
c=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

In [512]:
a['bagb'].mean()

0.5034863945578232

In [513]:
b['bagb'].mean()

0.5394557823129251

# Gradient Boosting Classifier


In [514]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')
c=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

In [516]:
a['gradb'].mean()

0.4739795918367347

In [517]:
b['gradb'].mean()

0.536734693877551

# Random Forest

In [518]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=20)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf')

In [519]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, rdf,'rdf')

In [520]:
a['rdf'].mean()

0.5062925170068027

In [521]:
b['rdf'].mean()

0.486139455782313

[<__main__.MyDf instance at 0x10fdcbdd0>,
 <__main__.MyDf instance at 0x10fe925a8>,
 <__main__.MyDf instance at 0x10fea92d8>,
 <__main__.MyDf instance at 0x10d78a488>,
 <__main__.MyDf instance at 0x10df0fa28>,
 <__main__.MyDf instance at 0x10d6077e8>,
 <__main__.MyDf instance at 0x10eb7a368>,
 <__main__.MyDf instance at 0x10fe9ad88>,
 <__main__.MyDf instance at 0x10e284b90>,
 <__main__.MyDf instance at 0x10dd345a8>,
 <__main__.MyDf instance at 0x10fe9ab00>,
 <__main__.MyDf instance at 0x10dd34290>,
 <__main__.MyDf instance at 0x10feb6710>,
 <__main__.MyDf instance at 0x10d60c998>,
 <__main__.MyDf instance at 0x10d5f79e0>,
 <__main__.MyDf instance at 0x10e295fc8>,
 <__main__.MyDf instance at 0x102a135a8>,
 <__main__.MyDf instance at 0x10e345680>]

# Working on sureness

In [688]:
df=allchampionships[0].df
df=df[df['season']==season_to_be_predicted]


In [690]:
for i in range(7,30):
    X_train=df[df.H_ROUND<i].iloc[:,column_to_use_as_prediction]
    Y_train=df[df.H_ROUND<i][column_to_predict]
    X_test=df[df.H_ROUND==i].iloc[:,column_to_use_as_prediction]
    Y_test=df[df.H_ROUND==i][column_to_predict]
    svm_linear = svm.SVC(kernel='linear')
    svm_linear.fit(X_train,Y_train)
    print(svm_linear.decision_function(X_test))
    #print(svm_linear.predict(X_test))
    #print(svm_linear.score(X_test,Y_test))
    #print(Y_test)
    

[ 1.87856284 -0.09564611  2.12047615  0.78856173  2.03696966]
[-0.24723744  0.57902468 -0.71624039 -0.65993478  1.97664831]
[-0.8210163  -1.31304309  1.54752469  2.51948387 -0.84971405]
[ 0.30952181  0.31552484 -1.31189154  0.33933124  0.32002133]
[-1.54280449 -0.57529349  0.72344877  0.23697091 -0.54589583]
[ 0.03253938  1.71403013 -0.39017587  1.31844906 -0.36300993]
[-0.08039024 -0.75336127  1.12289696 -0.07468573  0.75718523]
[ 1.18468021  0.7939626   0.91468621  1.27926597  0.56954476]
[ 0.99972309  0.99909338  1.00020599  0.99984165  0.99924218]
[ 0.99996415  0.99996507  0.99986976  1.00030334  0.99944997]
[ 0.99991859  1.00007532  0.99996626  0.99989313  0.999864  ]
[ 0.99957882  1.00011077  0.99995905  1.00028992  0.99977055]
[ 0.99931449  1.0001169   1.00033339  0.99921575  0.99995206]
[ 0.99861339  0.9996767   0.99918097  0.99952983]
[ 0.99987042  1.00013624  0.99982234  0.99956633  0.99997466  1.00014378
  1.0001255 ]
[ 0.6918021   0.79359592  0.86954444  1.00076569  1.04416

In [699]:
column_to_predict='FTR'

In [702]:
X_train=df[df.H_ROUND<30].iloc[:,column_to_use_as_prediction]
Y_train=df[df.H_ROUND<30][column_to_predict]
X_test=df[df.H_ROUND==31].iloc[:,column_to_use_as_prediction]
Y_test=df[df.H_ROUND==31][column_to_predict]
svm_linear = svm.SVC(kernel='linear', probability=True)
svm_linear.fit(X_train,Y_train)
print(svm_linear.predict_proba(X_test))
print(svm_linear.predict(X_test))
print(svm_linear.score(X_test,Y_test))
print(Y_test)


[[ 0.37541441  0.23005023  0.39453537]
 [ 0.20398232  0.25211461  0.54390308]
 [ 0.38333283  0.24017072  0.37649645]
 [ 0.2695469   0.26330741  0.46714568]
 [ 0.24472161  0.26556783  0.48971057]]
['A' 'H' 'A' 'H' 'H']
0.6
1392    A
1393    H
1394    D
1395    A
1396    H
Name: FTR, dtype: object


In [349]:
svm_linear.decision_function(X_test)

array([ 0.70918747,  0.66477063,  0.71625791,  0.75144355,  0.95903544])

In [350]:
svm_linear.predict(X_test)

array([1, 1, 1, 1, 1])

In [351]:
svm_linear.score(X_test,Y_test)

0.80000000000000004

In [316]:
Y_test

1387    1
1388    0
1389    0
1390    0
1391    0
Name: Goal/No_Goal, dtype: int64

In [345]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
rdf.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [346]:
rdf.feature_importances_

array([ 0.08142444,  0.09222371,  0.19321762,  0.17049799,  0.17541312,
        0.14064783,  0.14657529])

In [347]:
rdf.predict(X_test)

array([0, 1, 0, 1, 1])